# TERA - DSCSP - Aula 28
## Topic Analysis - Case (Solução)

## Case

Esse case foi retirado desse [artigo](https://towardsdatascience.com/topic-modeling-for-the-new-york-times-news-dataset-1f643e15caac). O objetivo dele é verificar se conseguimos extrair tópicos relevantes de um dataset contendo 8.447 matérias do NY Times, com um vocabulário de 3.012 palavras. Por razão de direitos autorias, os documentos não possuem título.

<img src="https://cdn-images-1.medium.com/max/1400/1*toWf7lAVf_5GIb9IMfS8Bw.jpeg" alt="Drawing" style="width: 800px;"/>


O arquivo `nyt_data.txt` contém os documentos, onde cada linha representa um documento específico. O dataset já sofreu um processo de limpeza e vetorização, onde foram mantidas apenas palavras que ocorreram mais de 10 vezes. Cada palavra é representada por um índice, que pode ser acessado pelo arquivo `nyt_vocab.txt`, e por sua frequência no documento.

Vamos criar a matriz de frequências a partir desses dados:

In [259]:
# Contém os documentos do dataset:
# Cada linha representa um documento
# Cada documento contém um conjunto de índices de palavras e 
# a sua frequência no documento -> word_idx:term_freq
with open(os.path.join(DATASET_FOLDER, 'nyt_data.txt')) as f:
    documents = f.readlines()
documents = [x.strip().strip('\n').strip("'") for x in documents] 

# Contém o vocabulário do dataset:
# Cada linha representa o índice da palavra
with open(os.path.join(DATASET_FOLDER, 'nyt_vocab.txt')) as f:
    vocabs = f.readlines()
vocabs = [x.strip().strip('\n').strip("'") for x in vocabs]

Vamos visualizar um exemplo de documento e vocabulários:

In [265]:
print("Documento 0:\n", documents[0])
print("\nParte do vocabulário:\n", vocabs[:10])

Documento 0:
 1946:2,1168:2,1194:2,1275:1,777:1,522:1,107:1,839:2,424:2,2330:2,1878:2,344:1,1008:1,94:3,735:1,212:1,2407:1,2623:1,781:2,42:1,160:1,1141:1,117:1,16:1,108:1,153:2,1137:2,416:1,23:2,46:1,734:2,284:1,207:2,301:1,357:1,780:2,2564:2,206:1,106:2,892:1,272:1,1557:1,2003:1,1079:2,370:1,1894:2,266:1,143:1,1532:1,2551:1,223:2,30:1,1509:1,1921:2,361:2,311:1,1549:2,203:1,2821:1,546:2,1612:1,200:1,247:2,1731:2,565:1,2253:1,234:1,72:1,648:1,1072:1,518:1,39:1,703:2,625:1,140:1,2301:1,32:1,462:1,743:1,2017:2,925:1,118:1,1000:2,836:1,161:1,942:2,885:1,267:1,2683:1,626:1,317:1,69:1,860:2,633:1,2658:1,75:2,1563:1,690:1,802:1,1650:1,1836:1,111:1,2151:1,128:2,2864:1,22:2,1301:1,250:2,1922:2,936:1,918:1,775:1,280:1,18:2,182:1,667:1,2383:1,878:1,1498:1,109:2,1577:1,1758:1,60:1,2269:1,215:1,2038:2,485:1,335:1,2543:2,422:1,12:1,585:2,1754:1,1293:1,604:1,52:1,248:1,2:2,603:2,1345:2,271:3,668:1,1022:2,260:1,1251:2,498:1,2213:1,351:3,2584:1,1349:1,334:2,218:1,34:1,256:1,2576:2,58:1,1637:1,926:2,313

Vamos agora criar a matriz de frequência de termos **A** a partir dos documentos.

In [270]:
# Número de documentos
n_doc = 8447
# Número de palavras
n_words = 3012 
A = np.zeros([n_doc,n_words])

for j in range(len(documents)):
    for i in documents[j].split(','):
        word, freq = i.split(':')
        A[j,int(word)-1] = int(freq)
        
# Tamanho da matriz de frequência de termos:
print(A.shape)

(8447, 3012)


Agora estamos prontos para realizar o processo de topic analysis! Divirta-se!

*Dica: Utilize entre 20 e 25 tópicos*

In [276]:
# Primeiro, importe o módulo NMF do scikit-learn
from sklearn.decomposition import NMF

# Precisamos criar a instância do NMF
# Vamos escolher 25 tópicos
n_topics = 25
nmf = NMF(n_components=n_topics)

# Agora vamos utilizar os mesmos atributos fit, transform ou fit_transform
# que já conhecemos do universo do sklearn
W_nmf = nmf.fit_transform(A)

# Vamos ver qual é a dimensão de W_nmf
W_nmf.shape

(8447, 25)

Note que o número de linhas se manteve em 8447, que é o número de documentos (artigos) que nós temos, e o número de colunas se transformou em 25, que é o número de tópicos que nós escolhemos. Essa matriz gerada representa a matriz **W** (matriz de pesos) da fatoração de matrizes.

Vamos agora achar a matriz **H** que representa a matriz de atributos.

In [277]:
H_nmf = nmf.components_
H_nmf.shape

(25, 3012)

Note que o número de linhas é igual ao número de tópicos e o número de colunas representa o número de palavras no nosso vocabulário. Cada linha da matriz é definida como um componente (assim como o PCA possui os componentes principais) que está associado a um tópico específico. Entretanto, diferentemente do PCA, nós podemos associar cada componente a um conjunto específico de palavras. Vamos verificar abaixo:

In [278]:
# Precisamos criar uma lista de palavras que representam as 
# colunas da matriz de frequência de palavras
words = vocabs

# Vamos criar um dataframe para visualizar
components_df = pd.DataFrame(nmf.components_, columns=words)

# Vamos verificar as palavras que representam mais fortemente cada um dos tópicos
for i in range(n_topics):
    component = components_df.iloc[i]
    print("Topico {}:".format(i))
    print("----------")
    print(component.nlargest(10))
    print("----------\n")

Topico 0:
----------
question     3.871823
thing        3.100133
point        2.938836
problem      2.428964
issue        2.416854
change       2.412662
believe      2.399133
mean         2.317026
public       2.179699
different    2.172736
Name: 0, dtype: float64
----------

Topico 1:
----------
thing     3.250954
tell      2.853146
ask       2.684436
feel      2.199998
lot       1.947914
little    1.926030
start     1.801750
room      1.640593
keep      1.610485
really    1.577409
Name: 1, dtype: float64
----------

Topico 2:
----------
company     6.297112
business    2.608162
sell        2.521198
share       2.332406
stock       2.245064
buy         2.008349
market      1.848729
sale        1.670624
large       1.559963
industry    1.463872
Name: 2, dtype: float64
----------

Topico 3:
----------
game       4.641119
play       3.908135
team       3.753881
season     3.225454
win        3.129571
second     2.980308
player     2.878549
victory    2.227106
score      2.101710
point   

Podemos perceber que os tópicos gerados são coerentes! Poderíamos facilmente associar cada um dos tópicos a um tema específico de uma matéria de jornal. Perceba o quanto isso é poderoso!

Podemos ainda verificar quais são os tópicos principais de alguns artigos específicos.

In [301]:
# Precisamos criar um dataframe para facilitar nossa vida
df = pd.DataFrame(data=W_nmf.T, 
                  columns=['doc {}'.format(i) for i in range(n_doc)], 
                  index=['topic {}'.format(i) for i in range(n_topics)])
df.loc[:,'doc 0':'doc 5'].head(25)

,doc 0,doc 1,doc 2,doc 3,doc 4,doc 5
topic 0,0.138997,0.112087,0.062792,0.324801,0.020204,0.000000
topic 1,0.000000,0.193594,0.004431,0.000000,0.188557,0.000000
topic 2,0.000000,0.017124,0.000000,0.098219,0.000000,0.030034
topic 3,0.000000,0.006128,0.000000,0.020459,0.007947,0.000000
topic 4,0.000962,0.049269,0.000000,0.074578,0.040884,0.000000
topic 5,0.000000,0.099079,0.000000,0.000000,0.099410,0.036020
topic 6,0.000000,0.056428,0.000000,0.051634,0.034271,0.010995
topic 7,0.143605,0.000000,0.000000,0.013844,0.000000,0.000000
topic 8,0.244761,0.007141,0.000000,0.276004,0.011112,0.000000
topic 9,0.000000,0.036408,0.000000,0.019394,0.044359,0.000000


Podemos notar que os documentos são compostos por diversos tópicos diferentes. Isso faz todo o sentido se pensarmos que as notícias não são tão específicas ao ponto de falar apenas de um único tema.

## Sistema de recomendação

Em poucos anos a internet revolucionou o mercado de consumo mundial e a forma como os clientes interagem com os vendedores. Uma das dinâmicas criadas mais importantes a partir dessa revolução é a de sistemas de recomendação. Todos devem já ter notado o quanto e-commerces como a Amazon, Best Buy e até empresas brasileiras acertam ao recomendar certos tipos de produto para seus clientes, que muitas vezes nem os estavam procurando (veja esse [artigo](https://www.techemergence.com/use-cases-recommendation-systems/)). Isso não acontece mais apenas em e-commerces, mas também com provedores de música (Spotify), de filmes (Netflix) ou vídeos em geral (Youtube). 

Esse fenômeno só ocorre devido a um fator: **dado**. As empresas atualmente possuem muita informação sobre os seus produtos e os seus usuários. É muito fácil obter, hoje em dia, os interesses dos clientes sobre seus produtos. Seja o fato de o vendedor comprar um produto, dar um review ou apenas clicar, já é suficiente para uma empresa mapear os interesses dos usuários e tentar direcionar produtos que seriam relevantes para o usuário sem nem mesmo ele saber!

Os sistemas de recomendação se baseiam, basicamente, em encontrar relações entre compradores e produtos. Mais especificamente, existem dois grandes grupos de sistemas de recomendação:
- **Proximidade de produtos**: Tem o objetivo de encontrar produtos similares aos consumidos por um cliente. Se um cliente possui o interesse em um determinado produto, o sistema de recomendação pode tentar encontrar outros produtos similares para indicar para o cliente.
- **Proximidade entre clientes** (Filtro Colaborativo): Tem o objetivo de encontrar clientes com interesses semelhantes. Suponha que exista um cliente X que consuma os produtos A e B, enquanto um outro cliente Y tem interesse nos produtos A, B e C. Como eles possuem interesses semelhantes (produtos A e B), o sistema de recomendação poderia indicar o produto C para o cliente X.

Existem diversos outros tipos de sistemas de recomendação que fogem do escopo desse material. Mais informações podem ser vistas nesse [link](https://www.techemergence.com/use-cases-recommendation-systems/).

##### Exemplo
Agora vamos tentar criar um sistema simples de recomendação baseado em **proximidade de produtos**! 

Vamos utilizar o dataset de artigos do NY Times para essa tarefa. Vamos supor que uma pessoa tenha lido um determinado artigo e nós gostaríamos de recomendar outros artigos semelhantes àquele. Lembre que para comparar dois documentos que contém vetores de atributos associados a palavras, a melhor medida de distância é a **distância de cossenos**.

Para calcular a distância entre produtos, nós poderíamos utilizar todo o espaço de atributos (quantidade de palavras no vocabulário), mas isso é desnecessário. Nós temos uma representação resumida de cada documento por sua proporção de tópicos, que formam o novo vetor de atributos. Como já encontramos os tópicos relacionados aos documentos no exercício anterior, podemos aproveitá-los para resolver nosso problema atual.

In [311]:
# Nós vamos ter que normalizar o vetor de atributos
# Isso é necessário para que todas as features (tópicos) 
# de um documento some 1 ao final
# Seria a porcentagem de composição dos tópicos 
# no documento
from sklearn.preprocessing import normalize
W_nmf_norm = normalize(W_nmf)

In [321]:
# Vamos criar um dataframe:
# índice: documento
# colunas: vetor de features (normalizadas)
df = pd.DataFrame(W_nmf_norm,
                  columns=['topic {}'.format(i) for i in range(n_topics)],
                  index=['doc {}'.format(i) for i in range(n_doc)])
df.head(5)

,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8,topic 9,...,topic 15,topic 16,topic 17,topic 18,topic 19,topic 20,topic 21,topic 22,topic 23,topic 24
doc 0,0.294146,0.000000,0.000000,0.000000,0.002036,0.000000,0.000000,0.303896,0.517961,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.082041,0.000000,0.000000,0.000000,0.153861,0.025917
doc 1,0.316441,0.546551,0.048343,0.017302,0.139095,0.279719,0.159308,0.000000,0.020161,0.102785,...,0.000000,0.012730,0.000000,0.000000,0.095460,0.057694,0.000000,0.000000,0.653904,0.018420
doc 2,0.524771,0.037032,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.791191,0.040254,0.000000,0.000000,0.161058,0.000000,0.000000,0.217096,0.000000,0.000000
doc 3,0.431515,0.000000,0.130489,0.027181,0.099081,0.000000,0.068598,0.018393,0.366686,0.025766,...,0.117798,0.002651,0.034564,0.024858,0.003601,0.070738,0.150782,0.154748,0.200475,0.000000
doc 4,0.024630,0.229870,0.000000,0.009689,0.049841,0.121191,0.041780,0.000000,0.013547,0.054078,...,0.053154,0.225243,0.084851,0.000000,0.130305,0.000000,0.421757,0.000000,0.000000,0.092667


In [314]:
# Vamos escolher um documento aleatoriamente
# Esse documento será o escolhido pelo cliente
doc_target = np.random.choice(range(len(documents)))
doc_target

7618

In [325]:
# Verifica o vetor de features dele:
doc_target_features = df.iloc[doc_target]
print(doc_target_features)

topic 0     0.133103
topic 1     0.575644
topic 2     0.051933
topic 3     0.000000
topic 4     0.550594
topic 5     0.025231
topic 6     0.424973
topic 7     0.139334
topic 8     0.206514
topic 9     0.107456
topic 10    0.000000
topic 11    0.000000
topic 12    0.161399
topic 13    0.000000
topic 14    0.000000
topic 15    0.000000
topic 16    0.140682
topic 17    0.000000
topic 18    0.000000
topic 19    0.000000
topic 20    0.000000
topic 21    0.135645
topic 22    0.154468
topic 23    0.000000
topic 24    0.046040
Name: doc 7618, dtype: float64


Encontre agora os documentos mais próximos a esse documento:

In [335]:
# Calcula a distância de cossenos entre o artigo e 
# todos os outros documentos
cossine_distance = df.dot(doc_target_features)

# Todos os documentos com maior distância de cossenos 
# representam os documentos mais próximos
print("Artigos recomendados:")
recommendations = cossine_distance.nlargest(6)[1:]
print(recommendations)

Artigos recomendados:
doc 1313    0.911258
doc 3083    0.905729
doc 4974    0.901671
doc 3370    0.883303
doc 5165    0.870204
dtype: float64


Vamos visualizar os principais tópicos desses documentos:

In [392]:
default_top_topics = df.iloc[doc_target].nlargest(3).index.values
default_components = [components_df.iloc[int(i.split(' ')[1])].nlargest(5).index.values 
                      for i in default_top_topics]

print("Artigo original: doc {}".format(doc_target))
print("-----------------------")
print("- Tópicos (palavras): ")
for i in range(len(top_topics)):
    print("{:>15} ({})".format(default_top_topics[i], ', '.join(default_components[i])))
print("\n\n")

for doc, similarity in recommendations.items():
    doc_num = int(doc.split(' ')[1])
    top_topics = df.iloc[doc_num].nlargest(3).index.values
    components = [components_df.iloc[int(i.split(' ')[1])].nlargest(5).index.values for i in top_topics]
    print("Recomendação {}:".format(doc))
    print("-----------------------")
    print("- Similaridade: {:.2%}".format(similarity))
    print("- Tópicos (palavras): ")
    for i in range(len(top_topics)):
        print("{:>15} ({})".format(top_topics[i], ', '.join(components[i])))
    print("\n")

Artigo original: doc 7618
-----------------------
- Tópicos (palavras): 
        topic 1 (thing, tell, ask, feel, lot)
        topic 4 (city, building, area, build, house)
        topic 6 (pay, money, state, budget, tax)



Recomendação doc 1313:
-----------------------
- Similaridade: 91.13%
- Tópicos (palavras): 
        topic 1 (thing, tell, ask, feel, lot)
        topic 6 (pay, money, state, budget, tax)
        topic 4 (city, building, area, build, house)


Recomendação doc 3083:
-----------------------
- Similaridade: 90.57%
- Tópicos (palavras): 
        topic 1 (thing, tell, ask, feel, lot)
        topic 4 (city, building, area, build, house)
        topic 6 (pay, money, state, budget, tax)


Recomendação doc 4974:
-----------------------
- Similaridade: 90.17%
- Tópicos (palavras): 
        topic 1 (thing, tell, ask, feel, lot)
       topic 22 (art, artist, exhibition, museum, painting)
        topic 6 (pay, money, state, budget, tax)


Recomendação doc 3370:
-----------------

Podemos perceber que os tópicos são realmente semelhantes. Poderíamos recomendar esses produtos para o cliente!